In [1]:
import utils
import duckdb
import numpy as np
import pandas as pd
import multiprocessing
import model as model
from datetime import datetime, date
import matplotlib.pyplot as plt

# Global vars
granularity = 'SME'
seasonality_factor = 0.4
agg_mode = model.agg_mode['p70']

# Fetch Data and Preprocessing

## Fetch Data

In [2]:
# Check occ old
utils.fetch_old_data("occ_old.csv", "data", utils.fetch_data)

'occ_old.csv' already exists in 'data'. Skipping fetch.


'data/occ_old.csv'

In [3]:
# Get occ new
utils.fetch_new_data()

'done'

## Ruas look up

In [5]:
data = pd.read_csv("data/occ_all.csv", low_memory=False)
data.createddate = pd.to_datetime(data.createddate)

In [6]:
data.head()

,node,ip,createddate,reg,metro,cap,traffic,mode
0,GPON00-D4-MGE-4FRAK,172.29.236.230,2024-03-31,4,ME-D4-MGL,20.0,2.6,uplink
1,GPON00-D4-MGE-4FRAK,172.29.236.230,2024-03-30,4,ME-D4-MGL,20.0,2.6,uplink
2,GPON00-D4-MGE-4FRAK,172.29.236.230,2024-03-29,4,ME-D4-MGL,20.0,2.8,uplink
3,GPON00-D4-MGE-4FRAK,172.29.236.230,2024-03-28,4,ME-D4-MGL,20.0,2.8,uplink
4,GPON00-D4-MGE-4FRAK,172.29.236.230,2024-03-27,4,ME-D4-MGL,20.0,2.8,uplink


In [7]:
df = utils.fill_ruas(data)
df['ruas'] = df['ruas'].str.strip()

### Checking

In [10]:
df['ruas'].nunique()

8900

In [11]:
df.dtypes

createddate    datetime64[ns]
reg                     int64
node                   object
metro                  object
ruas                   object
traffic               float64
cap                   float64
sumber                 object
mode                   object
dtype: object

In [12]:
df.isna().sum()

createddate         0
reg                 0
node                0
metro               0
ruas                0
traffic             0
cap                 0
sumber         557445
mode                0
dtype: int64

In [13]:
df['reg'].value_counts()

reg
5    1166790
2    1043345
1    1021936
7     683883
4     647042
6     633794
3     557750
0        310
Name: count, dtype: int64

In [14]:
df.createddate.min(), df.createddate.max()

(Timestamp('2022-06-13 00:00:00'), Timestamp('2025-04-22 00:00:00'))

In [15]:
df.to_csv("data/raw.csv", index=None)

In [16]:
df.head()

,createddate,reg,node,metro,ruas,traffic,cap,sumber,mode
0,2025-04-22,1,GPON00-D1-AGR-2,ME-D1-AGRA,GPON00-D1-AGR-2_to_ME-D1-AGRA,0.00,3.0,lldp,uplink_n
1,2025-04-22,1,GPON00-D1-AMK-2,ME-D1-AMKA,GPON00-D1-AMK-2_to_ME-D1-AMKA,0.00,1.0,lldp,uplink_n
2,2025-04-22,1,GPON00-D1-AMK-2UKUI,OTHER,GPON00-D1-AMK-2UKUI_to_ME-D1-RGTA,0.04,11.0,lldp,uplink_n
3,2025-04-22,1,GPON00-D1-ANR-2,ME-D1-ANRA,GPON00-D1-ANR-2_to_ME-D1-ANRA,0.00,3.0,lldp,uplink_n
4,2025-04-22,1,GPON00-D1-BAK-2,ME-D1-BAKA,GPON00-D1-BAK-2_to_ME-D1-BAKA,0.00,1.0,lldp,uplink_n


## Cut off Data

In [17]:
# Create a Pandas Timestamp object from the current date
current_date_timestamp = pd.Timestamp(datetime.now())

# Get the first day of the current month
first_day_of_month = current_date_timestamp.replace(
    day=1, hour=0, minute=0, second=0, microsecond=0
)

# Print the result
first_day_of_month

Timestamp('2025-04-01 00:00:00')

In [18]:
df = df[df.createddate < first_day_of_month]
df.createddate.max(), df.shape

(Timestamp('2025-03-31 00:00:00'), (5599163, 9))

### Resample and Save

In [19]:
df.reset_index(drop=True, inplace=True)
df.head()

,createddate,reg,node,metro,ruas,traffic,cap,sumber,mode
0,2025-03-31,2,GP0N00-D2-BJT-2PMR,OTHER,GP0N00-D2-BJT-2PMR_to_OTHER,0.06,10.0,NaN,uplink_n
1,2025-03-31,1,GPON00-D1-ABA-2,ME-D1-ABAA,GPON00-D1-ABA-2_to_ME-D1-ABAA,3.10,4.0,lldp,uplink_n
2,2025-03-31,1,GPON00-D1-ABG-2,OTHER,GPON00-D1-ABG-2_to_OTHER,2.61,11.0,NaN,uplink_n
3,2025-03-31,1,GPON00-D1-AGR-2,ME-D1-AGRA,GPON00-D1-AGR-2_to_ME-D1-AGRA,7.39,47.0,lldp,uplink_n
4,2025-03-31,1,GPON00-D1-AGR-2GRA,ME-D1-AGRA,GPON00-D1-AGR-2GRA_to_ME-D1-AGRA,0.20,10.0,descp,uplink_n


In [20]:
df_ = df[['createddate', 'reg', 'ruas', 'traffic']].copy()
df_.head()

,createddate,reg,ruas,traffic
0,2025-03-31,2,GP0N00-D2-BJT-2PMR_to_OTHER,0.06
1,2025-03-31,1,GPON00-D1-ABA-2_to_ME-D1-ABAA,3.10
2,2025-03-31,1,GPON00-D1-ABG-2_to_OTHER,2.61
3,2025-03-31,1,GPON00-D1-AGR-2_to_ME-D1-AGRA,7.39
4,2025-03-31,1,GPON00-D1-AGR-2GRA_to_ME-D1-AGRA,0.20


In [21]:
df_.dtypes

createddate    datetime64[ns]
reg                     int64
ruas                   object
traffic               float64
dtype: object

In [22]:
df_.to_csv("data/daily_data.csv", index=None)
dfmonthly_mean = (
    df_.groupby(
        [
            "ruas",
            pd.Grouper(key="createddate", freq="ME"),
        ],
        group_keys=True,
    )
    .mean()
    .reset_index()
)
dfmonthly_mean.to_csv("data/monthly_mean.csv", index=None)

In [23]:
df_agg = df_.groupby(
    ['ruas', pd.Grouper(key="createddate", freq=granularity)],
    group_keys=True
).agg({'traffic': agg_mode}).reset_index()

df_agg.to_csv("data/data_mean_mod.csv", index=None)

# Train Validation

In [24]:
df = pd.read_csv("data/data_mean_mod.csv")

df.createddate = pd.to_datetime(df.createddate)
df['yearmonth'] = df['createddate'].dt.strftime('%Y-%m')

df.head()

,ruas,createddate,traffic,yearmonth
0,09-MBS-35_to_ME-D2-KT2,2022-11-30,0.0,2022-11
1,09-MBS-35_to_ME-D2-KT2,2022-12-15,0.0,2022-12
2,09-MBS-35_to_ME-D2-KT2,2022-12-31,0.0,2022-12
3,09-MBS-35_to_ME-D2-KT2,2023-01-15,0.0,2023-01
4,09-MBS-35_to_ME-D2-KT2,2023-01-31,0.0,2023-01


## Remove old and new ruas

In [25]:
# Remove old ruas
old_ruas = utils.extract_old_ruas(df)

# Remove new ruas
new_ruas = utils.extract_new_ruas(df)

# Get forecast-able ruas
df_ = df[~df['ruas'].isin(old_ruas['ruas'])]
df_ = df_[~df_['ruas'].isin(new_ruas['ruas'])]

not_forecast_ruas = old_ruas['ruas'].to_list() + new_ruas['ruas'].to_list()

df['ruas'].nunique(), df_['ruas'].nunique()

(8741, 7420)

In [26]:
df_.head()

,ruas,createddate,traffic,yearmonth
60,BLADE-D2-LEGOK-2_to_OTHER,2024-07-31,0.00,2024-07
61,BLADE-D2-LEGOK-2_to_OTHER,2024-08-15,0.00,2024-08
62,BLADE-D2-LEGOK-2_to_OTHER,2024-08-31,0.00,2024-08
63,BLADE-D2-LEGOK-2_to_OTHER,2024-09-15,0.00,2024-09
64,BLADE-D2-LEGOK-2_to_OTHER,2024-09-30,0.01,2024-09


## Last value

In [27]:
# last value before modeling (only for old ruas)

df_lv = df[df['ruas'].isin(not_forecast_ruas)]
df_lv['ruas'].nunique()

1321

In [28]:
lv = utils.last_value(df_lv)

lv.head()

,ruas,createddate,traffic,yearmonth
0,09-MBS-35_to_ME-D2-KT2,2022-11-30,0.0,2022-11
1,09-MBS-35_to_ME-D2-KT2,2022-12-15,0.0,2022-12
2,09-MBS-35_to_ME-D2-KT2,2022-12-31,0.0,2022-12
3,09-MBS-35_to_ME-D2-KT2,2023-01-15,0.0,2023-01
4,09-MBS-35_to_ME-D2-KT2,2023-01-31,0.0,2023-01


In [29]:
# Combine last value with original df
df_ = pd.concat([df_, lv], axis=0, ignore_index=True)
df_['ruas'].nunique()

8741

### 1 month

#### Cut off Test Data

In [30]:
# Get the current date
current_date = datetime.today()

# Calculate the date 1 month ago
one_month_ago = current_date - pd.DateOffset(months=1)

# Get the first day of the month for the calculated date
START_TEST_DATE = one_month_ago.replace(day=1).strftime("%Y-%m-%d")
threshold_test = one_month_ago.replace(day=1).strftime("%Y-%m")

START_TEST_DATE, threshold_test

('2025-03-01', '2025-03')

In [31]:
df_train = df_[df_.createddate < START_TEST_DATE]

In [32]:
df_train['yearmonth'].value_counts()

yearmonth
2025-02    15901
2025-01    15860
2024-12    15679
2024-11    15609
2024-10    15541
2024-09    15461
2024-08    15280
2024-07    14597
2024-06    14419
2024-05    13976
2024-04    13918
2024-03    13274
2023-12    12210
2024-02    12197
2023-11    12128
2023-10    11992
2023-09    11919
2023-08    11863
2023-07    11840
2023-06    11824
2023-05    11818
2023-01    11470
2023-04    11462
2022-12    11183
2022-11    11157
2022-10    11130
2022-09    11057
2022-08    10906
2022-06    10833
2022-07    10805
2024-01     6218
2023-03     5793
2023-02     5788
2022-05     5361
Name: count, dtype: int64

#### Modeling

In [33]:
# model configuration
growth_factor = 0.2  # default: 0.2

In [34]:
# data for training and testing
df_train = utils.create_statsforecast_df(df_train)
df_test = utils.create_test_data(df_train, START_TEST_DATE, 0)

In [35]:
df_train.yearmonth.max(), df_test.yearmonth.min(), df_test.yearmonth.max()

('2025-02', '2025-03', '2025-05')

In [36]:
with multiprocessing.Pool() as a_pool:
     result = a_pool.starmap(
            model.predictions,
            [(ruas, df_train, df_test, granularity, agg_mode, growth_factor) \
             for ruas in df_train.unique_id.unique()]
)

In [37]:
result = pd.concat(result, ignore_index=True)
df_conslinreg = result[['unique_id', 'forecast', 'yearmonth']].rename(columns={'unique_id': 'ruas'}).copy()
df_conslinreg.to_csv("data/prediction_dataset.csv", index=None)

df_conslinreg.head()

,ruas,forecast,yearmonth
0,BLADE-D2-LEGOK-2_to_OTHER,0.007175,2025-03
1,BLADE-D2-LEGOK-2_to_OTHER,0.007292,2025-04
2,BLADE-D2-LEGOK-2_to_OTHER,0.007409,2025-05
3,GPON00-D1-ABA-2_to_ME-D1-ABAA,2.472978,2025-03
4,GPON00-D1-ABA-2_to_ME-D1-ABAA,2.521855,2025-04


#### Seasonality

In [38]:
# post-processing: add seasonality
df_conslinreg_deviation_agg = pd.concat([
    model.calculate_deviation_in_percent(
        df_train, ruas, granularity, agg_mode, growth_factor, seasonality_factor
    ).groupby(['ruas','month_name'], as_index=False).agg(avg_deviation=('deviation','mean')) \
    for ruas in df_conslinreg.ruas.unique()
])

df_conslinreg_seasonal = model.add_seasonality(
    df_conslinreg_deviation_agg, 
    df_conslinreg
)

df_conslinreg_seasonal.to_csv(f"data/{utils.LAYER}_validation_seasonality.csv", index=None)

#### Metrics

In [39]:
df_cur = pd.read_csv("data/daily_data.csv")
df_cur.drop('reg', axis=1, inplace=True)

df_cur.createddate = pd.to_datetime(df_cur.createddate)
df_cur['yearmonth'] = df_cur['createddate'].dt.strftime('%Y-%m')

df_cur.head()

,createddate,ruas,traffic,yearmonth
0,2025-03-31,GP0N00-D2-BJT-2PMR_to_OTHER,0.06,2025-03
1,2025-03-31,GPON00-D1-ABA-2_to_ME-D1-ABAA,3.10,2025-03
2,2025-03-31,GPON00-D1-ABG-2_to_OTHER,2.61,2025-03
3,2025-03-31,GPON00-D1-AGR-2_to_ME-D1-AGRA,7.39,2025-03
4,2025-03-31,GPON00-D1-AGR-2GRA_to_ME-D1-AGRA,0.20,2025-03


In [40]:
df_cur['yearmonth'].value_counts()

yearmonth
2025-03    226966
2025-01    222768
2024-12    222458
2024-10    220491
2024-08    215969
2024-11    214099
2024-09    211992
2024-07    207306
2024-05    206416
2024-06    200016
2024-04    199807
2025-02    195856
2023-12    179305
2024-01    179253
2023-07    174564
2023-09    174531
2023-05    171543
2023-06    170916
2022-10    169320
2022-12    168874
2023-01    166082
2022-08    165929
2022-07    164657
2023-08    163142
2022-11    159281
2023-11    153366
2023-02    146778
2023-10    146233
2023-04    129571
2024-03    107663
2022-09     81298
2022-06     58768
2024-02     23945
Name: count, dtype: int64

In [41]:
df_eval_conslinreg = utils.get_eval_data(
    df_cur, 
    df_conslinreg_seasonal, #df_conslinreg,
    threshold_test, 
    agg_mode
)

[INFO] jumlah unique ruas data actual: 7567
[INFO] jumlah unique ruas data prediction: 8612


In [42]:
metrics = utils.evaluation_model(
        y_actual=df_eval_conslinreg[['yearmonth','ruas','traffic']].query("yearmonth == @threshold_test"),
        y_pred=df_eval_conslinreg[['yearmonth','ruas','forecast']].query("yearmonth == @threshold_test"),
        layer=utils.LAYER_DASH,
        model=utils.MODEL_NAME,
        threshold=threshold_test,
        freq=granularity
)
    
#metrics.replace(np.inf, np.nan).to_sql("eval_train_model", utils.build_db_engine(), if_exists="append", index=None)
metrics

,smape,mape,wmape,r2_score,rmse,mae,layer,model,yearmonth,yearmonth_train_set,freq,os_username,insert_dma
0,0.035417,inf,0.06921,0.976908,0.801192,0.368584,olt - metro,ConsLinReg,202503,202502,SME,jupyter-896636@telkom.co.i-b8787,2025-04-24 18:05:14.933206


### 3 months

#### Cut off Test Data

In [41]:
# Get the current date
current_date = datetime.today()

# 1 month ago
months_ago = current_date - pd.DateOffset(months=1)
START_TEST_DATE_1 = months_ago.replace(day=1).strftime("%Y-%m-%d")
threshold_test_1 = months_ago.replace(day=1).strftime("%Y-%m")

# 2 months ago
months_ago = current_date - pd.DateOffset(months=2)
START_TEST_DATE_2 = months_ago.replace(day=1).strftime("%Y-%m-%d")
threshold_test_2 = months_ago.replace(day=1).strftime("%Y-%m")

# 3 months ago
months_ago = current_date - pd.DateOffset(months=3)
START_TEST_DATE_3 = months_ago.replace(day=1).strftime("%Y-%m-%d")
threshold_test_3 = months_ago.replace(day=1).strftime("%Y-%m")
threshold_test = months_ago.replace(day=1).strftime("%Y-%m")

threshold_test, threshold_test_1, threshold_test_2, threshold_test_3

('2025-01', '2025-03', '2025-02', '2025-01')

In [42]:
df_train = df_[df_.createddate < START_TEST_DATE_3]

In [43]:
df_train.yearmonth.min(), df_train.yearmonth.max()

('2022-05', '2024-12')

#### Modeling

In [44]:
# model configuration
growth_factor = 0.2  # default: 0.2

In [45]:
# data for training and testing
df_train = utils.create_statsforecast_df(df_train)
df_test = utils.create_test_data(df_train, START_TEST_DATE_3, 0)

In [46]:
df_train.yearmonth.max(), df_test.yearmonth.min(), df_test.yearmonth.max()

('2024-12', '2025-01', '2025-03')

In [47]:
with multiprocessing.Pool() as a_pool:
     result = a_pool.starmap(
            model.predictions,
            [(ruas, df_train, df_test, granularity, agg_mode, growth_factor) \
             for ruas in df_train.unique_id.unique()]
)

In [48]:
result = pd.concat(result, ignore_index=True)
df_conslinreg = result[['unique_id', 'forecast', 'yearmonth']].rename(columns={'unique_id': 'ruas'}).copy()

#### Seasonality

In [49]:
# post-processing: add seasonality
df_conslinreg_deviation_agg = pd.concat([
    model.calculate_deviation_in_percent(
        df_train, ruas, granularity, agg_mode, growth_factor, seasonality_factor
    ).groupby(['ruas','month_name'], as_index=False).agg(avg_deviation=('deviation','mean')) \
    for ruas in df_conslinreg.ruas.unique()
])

df_conslinreg_seasonal = model.add_seasonality(
    df_conslinreg_deviation_agg, 
    df_conslinreg
)

#### Metrics

In [50]:
df_cur = pd.read_csv("data/daily_data.csv")
df_cur.drop('reg', axis=1, inplace=True)

df_cur.createddate = pd.to_datetime(df_cur.createddate)
df_cur['yearmonth'] = df_cur['createddate'].dt.strftime('%Y-%m')

In [51]:
df_cur['yearmonth'].value_counts()

yearmonth
2025-03    226966
2025-01    222768
2024-12    222458
2024-10    220491
2024-08    215969
2024-11    214099
2024-09    211992
2024-07    207306
2024-05    206416
2024-06    200016
2024-04    199807
2025-02    195856
2023-12    179305
2024-01    179253
2023-07    174564
2023-09    174531
2023-05    171543
2023-06    170916
2022-10    169320
2022-12    168874
2023-01    166082
2022-08    165929
2022-07    164657
2023-08    163142
2022-11    159281
2023-11    153366
2023-02    146778
2023-10    146233
2023-04    129571
2024-03    107663
2022-09     81298
2022-06     58768
2024-02     23945
Name: count, dtype: int64

In [52]:
df_eval_conslinreg = utils.get_eval_data(
    df_cur, 
    df_conslinreg_seasonal, #df_conslinreg, 
    threshold_test, 
    agg_mode
)

[INFO] jumlah unique ruas data actual: 7614
[INFO] jumlah unique ruas data prediction: 8516


In [53]:
# last month
metrics = utils.evaluation_model(
        y_actual=df_eval_conslinreg[['yearmonth','ruas','traffic']].query("yearmonth == @threshold_test_1"),
        y_pred=df_eval_conslinreg[['yearmonth','ruas','forecast']].query("yearmonth == @threshold_test_1"),
        layer=utils.LAYER_DASH,
        model=utils.MODEL_NAME,
        threshold=threshold_test_1,
        freq=granularity,
        train_period=threshold_test_3
)
    
metrics.replace(np.inf, np.nan).to_sql("eval_train_model_t3", utils.build_db_engine(), if_exists="append", index=None)
metrics

,smape,mape,wmape,r2_score,rmse,mae,layer,model,yearmonth,yearmonth_train_set,freq,os_username,insert_dma
0,0.056063,inf,0.088941,0.96547,0.980572,0.478501,olt - metro,ConsLinReg,202503,202412,SME,jupyter-896636@telkom.co.i-b8787,2025-04-08 14:05:24.117884


In [54]:
# two months ago
metrics = utils.evaluation_model(
        y_actual=df_eval_conslinreg[['yearmonth','ruas','traffic']].query("yearmonth == @threshold_test_2"),
        y_pred=df_eval_conslinreg[['yearmonth','ruas','forecast']].query("yearmonth == @threshold_test_2"),
        layer=utils.LAYER_DASH,
        model=utils.MODEL_NAME,
        threshold=threshold_test_2,
        freq=granularity,
        train_period=threshold_test_3
)
    
metrics.replace(np.inf, np.nan).to_sql("eval_train_model_t3", utils.build_db_engine(), if_exists="append", index=None)
metrics

,smape,mape,wmape,r2_score,rmse,mae,layer,model,yearmonth,yearmonth_train_set,freq,os_username,insert_dma
0,0.042382,inf,0.081072,0.974715,0.8364,0.430621,olt - metro,ConsLinReg,202502,202412,SME,jupyter-896636@telkom.co.i-b8787,2025-04-08 14:05:28.488794


In [55]:
# three months ago
metrics = utils.evaluation_model(
        y_actual=df_eval_conslinreg[['yearmonth','ruas','traffic']].query("yearmonth == @threshold_test_3"),
        y_pred=df_eval_conslinreg[['yearmonth','ruas','forecast']].query("yearmonth == @threshold_test_3"),
        layer=utils.LAYER_DASH,
        model=utils.MODEL_NAME,
        threshold=threshold_test_3,
        freq=granularity,
        train_period=threshold_test_3
)
    
metrics.replace(np.inf, np.nan).to_sql("eval_train_model_t3", utils.build_db_engine(), if_exists="append", index=None)
metrics

,smape,mape,wmape,r2_score,rmse,mae,layer,model,yearmonth,yearmonth_train_set,freq,os_username,insert_dma
0,0.034752,inf,0.074731,0.980345,0.730952,0.391208,olt - metro,ConsLinReg,202501,202412,SME,jupyter-896636@telkom.co.i-b8787,2025-04-08 14:05:32.437410


# Forecast All Data

In [56]:
df = pd.read_csv("data/data_mean_mod.csv")

df.createddate = pd.to_datetime(df.createddate)
df['yearmonth'] = df['createddate'].dt.strftime('%Y-%m')

df.head()

,ruas,createddate,traffic,yearmonth
0,09-MBS-35_to_ME-D2-KT2,2022-11-30,0.0,2022-11
1,09-MBS-35_to_ME-D2-KT2,2022-12-15,0.0,2022-12
2,09-MBS-35_to_ME-D2-KT2,2022-12-31,0.0,2022-12
3,09-MBS-35_to_ME-D2-KT2,2023-01-15,0.0,2023-01
4,09-MBS-35_to_ME-D2-KT2,2023-01-31,0.0,2023-01


## Remove old and new ruas

In [57]:
# Remove old ruas
old_ruas = utils.extract_old_ruas(df)

# Remove new ruas
new_ruas = utils.extract_new_ruas(df)

# Get forecast-able ruas
df_ = df[~df['ruas'].isin(old_ruas['ruas'])]
df_ = df_[~df_['ruas'].isin(new_ruas['ruas'])]

not_forecast_ruas = old_ruas['ruas'].to_list() + new_ruas['ruas'].to_list()

df['ruas'].nunique(), df_['ruas'].nunique()

(8739, 7419)

## Last value

In [58]:
# last value before modeling (only for old ruas)

df_lv = df[df['ruas'].isin(not_forecast_ruas)]
df_lv['ruas'].nunique()

1320

In [59]:
# Combine last value with original df
lv = utils.last_value(df_lv)
df_ = pd.concat([df_, lv], axis=0, ignore_index=True)
df_['ruas'].nunique()

8739

## Modeling

In [60]:
# Create a Pandas Timestamp object from the current date
current_date_timestamp = pd.Timestamp(datetime.now())

# Get the first day of the current month
first_day_of_month = current_date_timestamp.replace(
    day=1, hour=0, minute=0, second=0, microsecond=0
)

# Print the result
first_day_of_month

Timestamp('2025-04-01 00:00:00')

In [61]:
# model configuration
growth_factor = 0.2  # default: 0.2

In [62]:
# data cut off
df_ = df_[df_.createddate < first_day_of_month]
df_.createddate.max()

Timestamp('2025-03-31 00:00:00')

In [63]:
# data for training and testing
df_train = df_.copy()
df_train = utils.create_statsforecast_df(df_train)

START_DATE = datetime.now().strftime("%Y-%m-01")
df_test = utils.create_test_data(df_train, START_DATE, 1)
START_DATE

'2025-04-01'

In [64]:
df_train.yearmonth.max(), df_test.yearmonth.min(), df_test.yearmonth.max()

('2025-03', '2025-04', '2026-04')

In [65]:
with multiprocessing.Pool() as a_pool:
     result = a_pool.starmap(
            model.predictions,
            [(ruas, df_train, df_test, granularity, agg_mode, growth_factor) \
             for ruas in df_train.unique_id.unique()]
)

In [66]:
result = pd.concat(result, ignore_index=True)
df_conslinreg = result[['unique_id', 'forecast', 'yearmonth']].rename(columns={'unique_id': 'ruas'}).copy()
df_conslinreg.to_csv("data/prediction_dataset_all.csv", index=None)

df_conslinreg.head()

,ruas,forecast,yearmonth
0,BLADE-D2-LEGOK-2_to_OTHER,3.529345e-08,2025-04
1,BLADE-D2-LEGOK-2_to_OTHER,5.882242e-08,2025-05
2,BLADE-D2-LEGOK-2_to_OTHER,8.235139e-08,2025-06
3,BLADE-D2-LEGOK-2_to_OTHER,1.176448e-07,2025-07
4,BLADE-D2-LEGOK-2_to_OTHER,1.411738e-07,2025-08


## Seasonality

In [67]:
# Get trend (growth)
df_conslinreg_deviation_agg = pd.concat([
    model.calculate_deviation_in_percent(
        df_train, ruas, granularity, agg_mode, growth_factor, seasonality_factor
    ).groupby(['ruas','month_name'], as_index=False).agg(avg_deviation=('deviation','mean')) \
    for ruas in df_conslinreg.ruas.unique()
])

# Compute seasonality
df_conslinreg_seasonal = model.add_seasonality(
    df_conslinreg_deviation_agg, 
    df_conslinreg
)

In [68]:
df_conslinreg.to_csv(f"data/{utils.LAYER}_before_seasonality.csv", index=None)
df_conslinreg_seasonal.to_csv(f"data/{utils.LAYER}_after_seasonality.csv", index=None)

# Save DB

In [69]:
# Get the current date
current_date = datetime.today()

# Calculate the date 1 month ago
one_month_ago = current_date - pd.DateOffset(months=1)

# Get the first day of the month for the calculated date
START_TEST_DATE = one_month_ago.replace(day=1).strftime("%Y-%m-%d")
threshold_test = one_month_ago.replace(day=1).strftime("%Y-%m")

START_TEST_DATE, threshold_test

('2025-03-01', '2025-03')

## Get capacity

In [70]:
raw = pd.read_csv("data/raw.csv")

raw.createddate = pd.to_datetime(raw.createddate)
raw['yearmonth'] = raw['createddate'].dt.strftime('%Y-%m')

raw.head()

,createddate,reg,node,metro,ruas,traffic,cap,sumber,mode,yearmonth
0,2025-04-07,2,GP0N00-D2-BJT-2PMR,OTHER,GP0N00-D2-BJT-2PMR_to_OTHER,0.05,10.0,NaN,uplink_n,2025-04
1,2025-04-07,1,GPON00-D1-AGR-2,ME-D1-AGRA,GPON00-D1-AGR-2_to_ME-D1-AGRA,0.00,1.0,lldp,uplink_n,2025-04
2,2025-04-07,1,GPON00-D1-AGR-3KRP,ME-D1-BNKC,GPON00-D1-AGR-3KRP_to_ME-D1-AGRA,0.00,10.0,lldp,uplink_n,2025-04
3,2025-04-07,1,GPON00-D1-AMK-2UKUI,OTHER,GPON00-D1-AMK-2UKUI_to_ME-D1-RGTA,0.00,10.0,lldp,uplink_n,2025-04
4,2025-04-07,1,GPON00-D1-ANR-2,ME-D1-ANRA,GPON00-D1-ANR-2_to_ME-D1-ANRA,0.00,3.0,lldp,uplink_n,2025-04


In [71]:
query = f"""
    SELECT ruas, reg as treg, cap as capacity
    FROM (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY ruas ORDER BY createddate DESC) as row_num
        FROM raw
        WHERE yearmonth <= '{threshold_test}'
    ) sub
    WHERE row_num = 1
    order by 1
"""

# Fetch the result as a DataFrame
dfcapacity = duckdb.query(query).to_df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [72]:
dfcapacity.head()

,ruas,treg,capacity
0,09-MBS-35_to_ME-D2-KT2,2,10.0
1,09-SPL-03_to_ME-D2-SPL,2,10.0
2,09PGG-12_to_ME-D2-PGG,2,21.0
3,172.21.212.239_to_OTHER,3,10.0
4,BLADE-D2-LEGOK-2_to_OTHER,2,1.0


## Processed output tables

In [73]:
df_final = pd.read_csv(f"data/{utils.LAYER}_after_seasonality.csv")
df_final.yearmonth = df_final.yearmonth.str.replace("-", "").astype(int)
df_final.head()

,ruas,forecast,yearmonth
0,BLADE-D2-LEGOK-2_to_OTHER,3.529345e-08,202504
1,BLADE-D2-LEGOK-2_to_OTHER,5.882242e-08,202505
2,BLADE-D2-LEGOK-2_to_OTHER,8.235139e-08,202506
3,BLADE-D2-LEGOK-2_to_OTHER,7.058690e-08,202507
4,BLADE-D2-LEGOK-2_to_OTHER,8.470428e-08,202508


In [74]:
df_validation = pd.read_csv(f"data/{utils.LAYER}_validation_seasonality.csv")
df_validation = df_validation[df_validation.yearmonth == threshold_test]
df_validation.yearmonth = df_validation.yearmonth.str.replace("-", "").astype(int)

df_validation.head()

,ruas,forecast,yearmonth
0,BLADE-D2-LEGOK-2_to_OTHER,0.007175,202503
3,GPON00-D1-ABA-2_to_ME-D1-ABAA,2.683435,202503
6,GPON00-D1-ABG-2_to_ME-D1-SPPA,2.631277,202503
9,GPON00-D1-ABG-2_to_OTHER,2.265606,202503
12,GPON00-D1-AGR-2_to_ME-D1-AGRA,8.827600,202503


In [75]:
df_final = pd.concat([df_validation, df_final], ignore_index=True)
df_final = df_final.sort_values(by=['ruas', 'yearmonth']).reset_index(drop=True)

df_final.head()

,ruas,forecast,yearmonth
0,09-MBS-35_to_ME-D2-KT2,0.0,202503
1,09-MBS-35_to_ME-D2-KT2,0.0,202504
2,09-MBS-35_to_ME-D2-KT2,0.0,202505
3,09-MBS-35_to_ME-D2-KT2,0.0,202506
4,09-MBS-35_to_ME-D2-KT2,0.0,202507


In [76]:
df_final_pred = pd.merge(df_final, dfcapacity, on="ruas", how="left")
df_final_pred = df_final_pred[['yearmonth', 'ruas', 'forecast', 'treg', 'capacity']]

df_final_pred.head()

,yearmonth,ruas,forecast,treg,capacity
0,202503,09-MBS-35_to_ME-D2-KT2,0.0,2,10.0
1,202504,09-MBS-35_to_ME-D2-KT2,0.0,2,10.0
2,202505,09-MBS-35_to_ME-D2-KT2,0.0,2,10.0
3,202506,09-MBS-35_to_ME-D2-KT2,0.0,2,10.0
4,202507,09-MBS-35_to_ME-D2-KT2,0.0,2,10.0


In [77]:
# tabel traffic
df_traffic = pd.pivot_table(
    df_final.loc[df_final['yearmonth'] != int(threshold_test.replace('-', ''))], values="forecast", columns="yearmonth", index=["ruas"]
).reset_index()

df_traffic = df_traffic.sort_values("ruas")
df_traffic.head()

yearmonth,ruas,202504,202505,202506,202507,202508,202509,202510,202511,202512,202601,202602,202603,202604
0,09-MBS-35_to_ME-D2-KT2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,09-SPL-03_to_ME-D2-SPL,4.695110e+00,4.830943e+00,4.845499e+00,4.807026e+00,4.848532e+00,4.822340e+00,4.855111e+00,4.952154e+00,4.937763e+00,5.071475e+00,5.096554e+00,4.959658e+00,4.889869e+00
2,09PGG-12_to_ME-D2-PGG,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,172.21.212.239_to_OTHER,1.000001e-02,1.000002e-02,1.000003e-02,1.000006e-02,1.000007e-02,1.000007e-02,1.000008e-02,1.000008e-02,1.000009e-02,1.000010e-02,1.000010e-02,1.000010e-02,1.000011e-02
4,BLADE-D2-LEGOK-2_to_OTHER,3.529345e-08,5.882242e-08,8.235139e-08,7.058690e-08,8.470428e-08,9.882166e-08,1.129390e-07,1.270564e-07,1.482325e-07,1.623499e-07,1.764673e-07,1.905846e-07,3.411700e-07


In [78]:
df_traffic_pred = pd.merge(df_traffic, dfcapacity, on="ruas", how="left")
columns = ['treg', 'ruas', 'capacity'] + [col for col in df_traffic_pred.columns if col not in ['treg', 'capacity', 'ruas']]

# Rearrange the DataFrame columns
df_traffic_pred = df_traffic_pred[columns]

df_traffic_pred.head()

,treg,ruas,capacity,202504,202505,202506,202507,202508,202509,202510,202511,202512,202601,202602,202603,202604
0,2,09-MBS-35_to_ME-D2-KT2,10.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,2,09-SPL-03_to_ME-D2-SPL,10.0,4.695110e+00,4.830943e+00,4.845499e+00,4.807026e+00,4.848532e+00,4.822340e+00,4.855111e+00,4.952154e+00,4.937763e+00,5.071475e+00,5.096554e+00,4.959658e+00,4.889869e+00
2,2,09PGG-12_to_ME-D2-PGG,21.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,3,172.21.212.239_to_OTHER,10.0,1.000001e-02,1.000002e-02,1.000003e-02,1.000006e-02,1.000007e-02,1.000007e-02,1.000008e-02,1.000008e-02,1.000009e-02,1.000010e-02,1.000010e-02,1.000010e-02,1.000011e-02
4,2,BLADE-D2-LEGOK-2_to_OTHER,1.0,3.529345e-08,5.882242e-08,8.235139e-08,7.058690e-08,8.470428e-08,9.882166e-08,1.129390e-07,1.270564e-07,1.482325e-07,1.623499e-07,1.764673e-07,1.905846e-07,3.411700e-07


In [79]:
df_final_cap = pd.merge(df_final.loc[df_final['yearmonth'] != int(threshold_test.replace('-', ''))], dfcapacity, on="ruas", how="left")
df_final_cap["percent_utilization"] = (df_final_cap.forecast / df_final_cap.capacity)*100

df_final_cap.head()

,ruas,forecast,yearmonth,treg,capacity,percent_utilization
0,09-MBS-35_to_ME-D2-KT2,0.0,202504,2,10.0,0.0
1,09-MBS-35_to_ME-D2-KT2,0.0,202505,2,10.0,0.0
2,09-MBS-35_to_ME-D2-KT2,0.0,202506,2,10.0,0.0
3,09-MBS-35_to_ME-D2-KT2,0.0,202507,2,10.0,0.0
4,09-MBS-35_to_ME-D2-KT2,0.0,202508,2,10.0,0.0


In [80]:
df_util = pd.pivot_table(
    df_final_cap,
    values="percent_utilization",
    columns="yearmonth",
    index=["ruas"],
    dropna=False,
).reset_index()

df_util.head()

yearmonth,ruas,202504,202505,202506,202507,202508,202509,202510,202511,202512,202601,202602,202603,202604
0,09-MBS-35_to_ME-D2-KT2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,09-SPL-03_to_ME-D2-SPL,46.951097,48.309428,48.454994,48.070264,48.485318,48.223403,48.551106,49.521544,49.377627,50.714749,50.965542,49.596576,48.898691
2,09PGG-12_to_ME-D2-PGG,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,172.21.212.239_to_OTHER,0.100000,0.100000,0.100000,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001
4,BLADE-D2-LEGOK-2_to_OTHER,0.000004,0.000006,0.000008,0.000007,0.000008,0.000010,0.000011,0.000013,0.000015,0.000016,0.000018,0.000019,0.000034


In [81]:
df_util_pred = pd.merge(df_util, dfcapacity, on="ruas", how="left")
columns = ['treg', 'ruas', 'capacity'] + [col for col in df_traffic_pred.columns if col not in ['treg', 'capacity', 'ruas']]

# Rearrange the DataFrame columns
df_util_pred = df_util_pred[columns]

df_util_pred.head()

,treg,ruas,capacity,202504,202505,202506,202507,202508,202509,202510,202511,202512,202601,202602,202603,202604
0,2,09-MBS-35_to_ME-D2-KT2,10.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,09-SPL-03_to_ME-D2-SPL,10.0,46.951097,48.309428,48.454994,48.070264,48.485318,48.223403,48.551106,49.521544,49.377627,50.714749,50.965542,49.596576,48.898691
2,2,09PGG-12_to_ME-D2-PGG,21.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,172.21.212.239_to_OTHER,10.0,0.100000,0.100000,0.100000,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001,0.100001
4,2,BLADE-D2-LEGOK-2_to_OTHER,1.0,0.000004,0.000006,0.000008,0.000007,0.000008,0.000010,0.000011,0.000013,0.000015,0.000016,0.000018,0.000019,0.000034


In [82]:
# save to csv
df_final_pred.to_csv(f"data/pred_{utils.LAYER}.csv", index=None)
df_traffic_pred.to_csv(f"data/pred_{utils.LAYER}_traffic.csv", index=None)
df_util_pred.to_csv(f"data/pred_{utils.LAYER}_util.csv", index=None)

In [83]:
# upload to db
from sqlalchemy.exc import ProgrammingError
engine = utils.build_db_engine()

min_yearmonth = str(df_final["yearmonth"].min())
min_yearmonth

'202503'

In [84]:
## delete old predictions
from sqlalchemy import text

query_drop = f"DELETE FROM pred_{utils.LAYER} WHERE yearmonth >= {min_yearmonth}"

try:
    with engine.connect() as con:
        con.execute(text(query_drop))
        con.commit()  # Explicit commit after deletion
    print("Rows deleted successfully.")
except Exception as e:
    print("An error occurred:", e)

Rows deleted successfully.


In [85]:
# tabel pred
df_final_pred.to_sql(
    f"pred_{utils.LAYER}", engine, if_exists="append", index=None
)

122154

In [86]:
# tabel util
df_util_pred.replace(np.inf, np.nan).to_sql(
    f"pred_{utils.LAYER}_util", engine, if_exists="replace", index=None
)

8734

In [87]:
# tabel traffic
df_traffic_pred.to_sql(
    f"pred_{utils.LAYER}_traffic", engine, if_exists="replace", index=None
)

8734

# Comp Table

## Cut off date

In [88]:
# Get 7 months ago from today --> 6 months ago from testing period

# Get the current date
current_date = datetime.today()

# Calculate the date 7 month ago
one_month_ago = current_date - pd.DateOffset(months=6)

# Get the first day of the month for the calculated date
comp_period = one_month_ago.replace(day=1).strftime("%Y%m")

comp_period

'202410'

In [89]:
# Get the current date
current_date = datetime.today()

# Calculate the date 1 month ago
one_month_ago = current_date - pd.DateOffset(months=1)

# Get the first day of the month for the calculated date
START_TEST_DATE = one_month_ago.replace(day=1).strftime("%Y-%m-%d")
threshold_test = one_month_ago.replace(day=1).strftime("%Y-%m")

START_TEST_DATE, threshold_test

('2025-03-01', '2025-03')

## Get capacity

In [90]:
raw = pd.read_csv("data/raw.csv")

raw.createddate = pd.to_datetime(raw.createddate)
raw['yearmonth'] = raw['createddate'].dt.strftime('%Y-%m')

query = f"""
    SELECT ruas, reg as treg, cap as capacity
    FROM (
        SELECT *,
               ROW_NUMBER() OVER (PARTITION BY ruas ORDER BY createddate DESC) as row_num
        FROM raw
        WHERE yearmonth <= '{threshold_test}'
    ) sub
    WHERE row_num = 1
    order by 1
"""

# Fetch the result as a DataFrame
dfcapacity = duckdb.query(query).to_df()

dfcapacity.head()

,ruas,treg,capacity
0,09-MBS-35_to_ME-D2-KT2,2,10.0
1,09-SPL-03_to_ME-D2-SPL,2,10.0
2,09PGG-12_to_ME-D2-PGG,2,21.0
3,172.21.212.239_to_OTHER,3,10.0
4,BLADE-D2-LEGOK-2_to_OTHER,2,1.0


In [91]:
# Load df actual
dfm = pd.read_csv("data/data_mean_mod.csv")
dfm["yearmonth"] = dfm.createddate.str[:7].str.replace("-", "").astype(int)

dfm.head()

,ruas,createddate,traffic,yearmonth
0,09-MBS-35_to_ME-D2-KT2,2022-11-30,0.0,202211
1,09-MBS-35_to_ME-D2-KT2,2022-12-15,0.0,202212
2,09-MBS-35_to_ME-D2-KT2,2022-12-31,0.0,202212
3,09-MBS-35_to_ME-D2-KT2,2023-01-15,0.0,202301
4,09-MBS-35_to_ME-D2-KT2,2023-01-31,0.0,202301


In [93]:
# Load df forecast
df_forecast = utils.pd_read_sql(
    f"select * from pred_{utils.LAYER} where yearmonth >= {comp_period} order by ruas, yearmonth",
)
df_forecast['yearmonth'] = df_forecast['yearmonth'].astype(int)

df_forecast.head()

,yearmonth,ruas,forecast,capacity,treg
0,202410,GPON00-D4-MGE-4FRAK_to_ME2-D4-MGL,1.166,20.0,4
1,202411,GPON00-D4-MGE-4FRAK_to_ME2-D4-MGL,0.000,20.0,4
2,202412,GPON00-D4-MGE-4FRAK_to_ME2-D4-MGL,0.000,20.0,4
3,202501,GPON00-D4-MGE-4FRAK_to_ME2-D4-MGL,0.000,20.0,4
4,202410,09-MBS-35_to_ME-D2-KT2,0.000,0.0,2


In [94]:
dfm['ruas'].nunique(), df_forecast['ruas'].nunique()

(8739, 9384)

In [95]:
# join df actual and forecast
df_join = pd.merge(dfm, df_forecast, on=["ruas", "yearmonth"])[
    ["ruas", "yearmonth", "traffic", "forecast"]
].rename(columns={"traffic": "actual"})

df_join["difference"] = df_join["actual"] - df_join["forecast"]
df_join['sape'] = [utils.sape(actual,forecast) for actual,forecast in zip(df_join['actual'], df_join['forecast'])]
df_join['sape'] = df_join.sape.astype(float)

df_join.head()

,ruas,yearmonth,actual,forecast,difference,sape
0,BLADE-D2-LEGOK-2_to_OTHER,202503,0.000,0.007175,-0.007175,0.000000
1,GP0N00-D2-BJT-2PMR_to_OTHER,202503,0.080,0.039690,0.040310,0.000000
2,GP0N00-D2-BJT-2PMR_to_OTHER,202503,0.060,0.039690,0.020310,0.000000
3,GPON00-D1-ABA-2_to_ME-D1-ABAA,202410,2.115,2.308036,-0.193036,0.043643
4,GPON00-D1-ABA-2_to_ME-D1-ABAA,202410,2.136,2.308036,-0.172036,0.038712


In [96]:
## write to table
df_join.to_sql(f"pred_{utils.LAYER}_comp_ruas", utils.build_db_engine(), if_exists="replace", index=None)

69505

In [97]:
# generate pivoted comp table
df_join["%dev"] = df_join["sape"] * 100
del df_join["sape"]

comp = pd.pivot_table(df_join, ["actual", "forecast", "%dev"], "ruas", "yearmonth", aggfunc=agg_mode)
comp_columns_list = comp.columns.to_flat_index().tolist()
comp_columns_list_list = [list(x) for x in comp_columns_list]
for x in comp_columns_list_list:
    x[1] = str(x[1])
    x[0], x[1] = x[1], x[0]

comp.columns = ["_".join(a) for a in comp_columns_list_list]
comp = comp.sort_index(axis=1)

comp.head()

,202410_%dev,202410_actual,202410_forecast,202411_%dev,202411_actual,202411_forecast,202412_%dev,202412_actual,202412_forecast,202501_%dev,202501_actual,202501_forecast,202503_%dev,202503_actual,202503_forecast
ruas,,,,,,,,,,,,,,,
BLADE-D2-LEGOK-2_to_OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0000,0.007175
GP0N00-D2-BJT-2PMR_to_OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0740,0.039690
GPON00-D1-ABA-2_to_ME-D1-ABAA,4.216379,2.1297,2.308036,6.784113,2.1638,2.472331,4.318394,2.5587,2.664670,6.969397,2.4588,2.778871,5.041903,2.9845,2.683435
GPON00-D1-ABG-2_to_ME-D1-SPPA,11.717254,2.1824,2.745462,4.138104,2.2618,2.450979,6.139361,2.3123,2.597078,3.653785,2.4295,2.598653,3.106498,2.8000,2.631277
GPON00-D1-ABG-2_to_OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,2.4250,0.000000,6.832992,2.5980,2.265606


In [98]:
comp_pred = pd.merge(comp, dfcapacity, on="ruas", how="left")
columns = ['treg', 'ruas', 'capacity'] + [col for col in comp_pred.columns if col not in ['treg', 'capacity', 'ruas']]

# Rearrange the DataFrame columns
comp_pred = comp_pred[columns]

comp_pred.head()

,treg,ruas,capacity,202410_%dev,202410_actual,202410_forecast,202411_%dev,202411_actual,202411_forecast,202412_%dev,202412_actual,202412_forecast,202501_%dev,202501_actual,202501_forecast,202503_%dev,202503_actual,202503_forecast
0,2,BLADE-D2-LEGOK-2_to_OTHER,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0000,0.007175
1,2,GP0N00-D2-BJT-2PMR_to_OTHER,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0740,0.039690
2,1,GPON00-D1-ABA-2_to_ME-D1-ABAA,4.0,4.216379,2.1297,2.308036,6.784113,2.1638,2.472331,4.318394,2.5587,2.664670,6.969397,2.4588,2.778871,5.041903,2.9845,2.683435
3,1,GPON00-D1-ABG-2_to_ME-D1-SPPA,12.0,11.717254,2.1824,2.745462,4.138104,2.2618,2.450979,6.139361,2.3123,2.597078,3.653785,2.4295,2.598653,3.106498,2.8000,2.631277
4,1,GPON00-D1-ABG-2_to_OTHER,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,2.4250,0.000000,6.832992,2.5980,2.265606


In [99]:
if comp_pred.shape[0] > 0:
    comp_pred.reset_index(drop=True).to_sql(
        f"pred_{utils.LAYER}_comp", utils.build_db_engine(), if_exists="replace", index=None
    )
    print("Done")

else:
    print("Didn't update the comp table because it was zero rows")

Done


# Validate total forecasted ruas

In [ ]:
df = pd.read_csv("data/raw.csv")

df.createddate = pd.to_datetime(df.createddate)
df['yearmonth'] = df['createddate'].dt.strftime('%Y-%m')

df.head()

In [ ]:
# Get the current date
current_date = datetime.today()
threshold = current_date.replace(day=1).strftime("%Y-%m")

threshold

In [ ]:
df[df['yearmonth'] == threshold]['ruas'].nunique()

In [ ]:
# Load df forecast
utils.pd_read_sql(
    f"SELECT count(distinct ruas) from pred_{utils.LAYER} where yearmonth = {int(threshold.replace('-', ''))}",
)